# Red River Bathymetry Inversion Example
This is a first cut at bathymetry inversion on a section of the Red River below Acme, LA.

The reach is roughly 9 [km] long. The cross-channel distances range approximately between 100 and 300 m. 


### Reference bathymetry
Our reference bathymetry is built by interpolating cross-sections from a detailed multibeam survey conducted by the ERDC Coastal and Hydraulics Lab. The problem domain and interpolated bathymetry on the a reference mesh are shown below.

<table>
<tr><th> Red River Domain</th> <th> Survey-Generated Bathymetry</th>
<tr>
<td> <img src="./plots/inset_google_map.png" alt="Red River Domain" style="width:300px"> </td>
<td> <img src="./plots/bathymetry_true.png" alt="Red River Bathymetry" style="width:300px"> </td>
</table>

## Auxiliary Conditions
Boundary conditions for the problem were a constant discharge $Q_b= 829.4$ [m${}^3$/s] at the inflow (upper left) boundary and a constant tailwater (free-surface elevation) $z_b=4.5$ [m] at the outflow (lower right).

The initial conditions were
$$
\mathbf{v}=0 \\
\eta = z_b
$$
where $\eta$ is the free-surface elevation, $\eta = z+h$, and $\mathbf{v}$ is the depth averaged velocity.

The simulations were run for 1 hour using the Corps' `Adaptive Hydraulics (AdH)` 2d shallow water module.

**Note** We will probably need to run the simulation longer or use a better initial condition from an offline 'spin-up' for the actual data collection.

### Velocity and Depth profiles
The velocity and depth predicted by AdH on the reference bathymetry are below. 

**TODO** replace these with plots in consistent color scheme with bathymetry plot

<table>
<tr><th> Velocity </th> <th> Depth</th>
<tr>
<td> <img src="./plots/velocity_inset_true.png" alt="Velocity for Collection Period" style="width:300px"> </td>
<td> <img src="./plots/depth_inset_true.png" alt="Depth for Collection Period" style="width:300px"> </td>
</table>

Import the necessary python modules

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter

import numpy as np
from pyPCGA import PCGA
from multiprocessing import Pool
import math
from scipy.io import savemat, loadmat
import adh


## Simulation parameters

Parameters controlling the `AdH` simulations are

| Option | Description |
|--------|-------------|
|`sim_dir` | where to run the simulations |
|`adh_exec`| path to the `AdH` executable   |
|`adh_version` | version of `AdH` being used|
|`adh_grid`    | `gridgen` grid file for logically rectangular domain|
|`adh_rect`    | `gridgen` bounding rectangle file |
|`adh_mesh`    | `AdH` mesh in `xms` format | 
|`adh_bc`      | `AdH` input file| 
|`nx`          | number of nodes in the stream-wise direction|
|`ny`          | number of nodes in the cross-channel direction|

**Note** $nx$ and $ny$ must be consistent with the mesh and grid information specified in the file parameters.


Parameters for the observations are

|Option |Description  |
|-------|-------------|
|`velocity_obs_file` | location of velocity observations |
|`elevation_obs_file`| location of elevation observations|
|`true_soln_file_h5` | `hdf5` file holding the reference solution (for synthetic cases)|
|`true_soln_meshbase`| basename for reference solution xms mesh


In [ ]:
adh_params = {'sim_dir': './simul',
              'adh_exec': './bin/v4/adh',
              'pre_adh_exec': './bin/v4/pre_adh',
              'adh_version': 4.5,
              'adh_grid': './mesh_files/nx1001_ny51/grid_Inset_nx1001_ny51',
              'adh_rect': './mesh_files/nx1001_ny51/rect_Inset_nx1001_ny51',
              'adh_mesh': './mesh_files/nx1001_ny51/Inset_nx1001_ny51.3dm',
              'adh_bc': './true_files/nx1001_ny51/Inset_true_v46.bc',
              'velocity_obs_file': './observation_files/observation_loc_N250_M8_J1_I10.dat',
              'elevation_obs_file': './observation_files/observation_loc_none.dat',
              'true_soln_file_h5': './true_files/nx1001_ny51/Inset_true_v46_p0.h5',
              'true_soln_meshbase': './true_files/nx1001_ny51/Inset_true_v46'
              }

Setup the domain and coordinate information

In [ ]:
nx,ny = 1001,51

N = np.array([nx, ny])
m = np.prod(N)
dx = np.array([1., 1.])

x = np.linspace(0., float(nx), N[0])
y = np.linspace(0., float(ny), N[1])

xmin = np.array([x[0], y[0]])
xmax = np.array([x[-1], y[-1]])

domain_output="""
Using nx,ny = {0},{1}.
xmin,xmax   = {2},{3}
"""

print(domain_output.format(nx,ny,xmin,xmax))

## Prior covariance

Specify the covariance standard deviation and length scales in terms of the logically rectangular reference domain (pixels)

In [ ]:
prior_std = 5.0
prior_cov_scale = np.array([40.,5.])

In [ ]:
def kernel(r): return (prior_std**2) * np.exp(-r**2)

### Setup the computational domain, initial estimate

The computational grid in the logically rectangular reference domain

In [ ]:
XX, YY = np.meshgrid(x, y)
pts = np.hstack((XX.ravel()[:, np.newaxis], YY.ravel()[:, np.newaxis]))

Use the a constaint value set to the mean for the true solution as the initial estimate for $z$.

In [ ]:
s_true = np.loadtxt('true.txt')
s_true = s_true.reshape(-1, 1)

In [ ]:
s_init = np.mean(s_true) * np.ones((m,1))

## The observations 

In [ ]:
obs = np.loadtxt('obs.txt')
obs = obs.reshape(-1, 1)

### The interface to the forward model

In [ ]:
def forward_model(s, parallelization, ncores=None):
    mymodel = adh.Model(adh_params)

    if parallelization:
        if ncores is None:
            from psutil import cpu_count  # physcial cpu counts
            ncores = cpu_count(logical=False)
        simul_obs = mymodel.run(s, parallelization, ncores)
    else:
        simul_obs = mymodel.run(s, parallelization)

    if simul_obs.ndim == 1:
        simul_obs = simul_obs.reshape(-1, 1)

    return simul_obs


## Inversion parameters

Parameters controlling `PCGA` are

|Option |Description |
|-------|------------|
|`R`    | observation variance    |
|`n_pc` | number of modes to use  |
|`max_iter`| max `PCGA` iterations|
|`restol`  | convergence tolerance|
|`matvec`  | matrix-vector product approach|
|          | `FFT`,`Dense`|
|`xmin`    | domain lower bounds|
|`xmax`    | domain upper bounds|
|`N`       | grid size|
|`prior_std` | prior standard deviation|
|`kernel`    | prior covariance kernel |
|`post_cov`  | compute posterior covariance?|
|`precond`   | use preconditioning?|
|`parallel`  | run simulations in parallel?|
|`LM`        | use Levenberg-Marquardt to solve Co-Kriging system?|
|`linesearch`| include line-search in Co-Kriging solution?|
|`forward_params`| dictionary of parameters to pass to forward model|
|`forward_model_verbose` | run model in verbose mode? |
|`verbose`   | run inversion in verbose mode?|
|`iter_save` | save intermediate iteration results?|

In [ ]:
params = {'R': (0.05) ** 2, 'n_pc': 100,
          'maxiter': 10, 'restol': 5e-2,
          'matvec': 'FFT', 'xmin': xmin,
          'xmax': xmax, 'N': N,
          'prior_std': prior_std, 'prior_cov_scale': prior_cov_scale,
          'kernel': kernel, 'post_cov': True, 'precond': True,
          'parallel': True, 'LM': True,
          'linesearch': True,
          'forward_params': adh_params,
          'forward_model_verbose': False, 'verbose': False,
          'iter_save': True
          }

## The inversion

First initialize the solver

In [ ]:
prob = PCGA(forward_model, s_init, pts, params, s_true, obs)

Run the inversion!

|Output | Description |
|-------|-------------|
|`s_hat`| PCGA estimate|
|`simul_obs` | observations obtained with `s_hat`|
|`post_diagv`| diagonal of posterior covariance |
|`iter_best` | iteration at which `s_hat` was found|



In [ ]:
s_hat, simul_obs, post_diagv, iter_best = prob.Run()

## Postprocess the results

Save some simulation details and results to disk

In [ ]:
savemat('results.mat', {'s_hat': s_hat, 'simul_obs': simul_obs,
                        'iter_best': iter_best,
                        'objvals': prob.objvals, 'R': prob.R,
                        'n_pc': prob.n_pc,'matvec':prob.matvec,
                        'prior_std': params['prior_std'], 'prior_cov_scale': params['prior_cov_scale'],
                        'LM': prob.LM, 'linesearch': prob.linesearch,
                        'Q2': prob.Q2_best, 'cR': prob.cR_best,
                        'maxiter': prob.maxiter, 'i_best': prob.i_best,
                        'restol': prob.restol, 'diagv': post_diagv})